In [ ]:
from coverage import *
from sgp4.api import Satrec, WGS72
from skyfield.api import load, EarthSatellite
import folium
import numpy as np

In [ ]:
delta_mos = np.arange(0, 361, 22.5)

# <EarthSatellite catalog #39084 epoch 2021-07-18 22:54:21 UTC>
times = gen_times(start_yr=2021, start_mo=7, start_day=18, days=1, step_min=0.22)
inst = camera_model(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, ) 

In [ ]:
degtorad = (np.pi/180)
# delta_mos = np.array([0., 90., 180., 270.])*degtorad
delta_mos = np.arange(0,360,22.5)*degtorad

In [ ]:
delta_mos

In [ ]:
sats = []
ts = load.timescale()

# for delta_mo in delta_mos:
for delta_mo in delta_mos:
    sat_ref = load.tle_file("./historical_tles/sat39084-tle1.txt")[0]
    satrec = Satrec()
    satrec.sgp4init(
        WGS72,           # gravity model
        'i',             # 'a' = old AFSPC mode, 'i' = improved mode
        sat_ref.model.satnum,               # satnum: Satellite number
        (50 + sat_ref.model.epochyr)*365.5 + sat_ref.model.epochdays,       # epoch: days since 1949 December 31 00:00 UT
        sat_ref.model.bstar,      # bstar: drag coefficient (/earth radii)
        sat_ref.model.nddot, # ndot: ballistic coefficient (revs/day)
        0.0,             # nddot: second derivative of mean motion (revs/day^3)
        sat_ref.model.ecco,       # ecco: eccentricity
        sat_ref.model.argpo, # argpo: argument of perigee (radians)
        sat_ref.model.inclo, # inclo: inclination (radians)
        sat_ref.model.mo + delta_mo, # mo: mean anomaly (radians)
        sat_ref.model.no_kozai, # no_kozai: mean motion (radians/minute)
        sat_ref.model.nodeo, # nodeo: right ascension of ascending node (radians)
    )
    sat_out = EarthSatellite.from_satrec(satrec, ts)
    sats.append(sat_out)

In [ ]:
sats[0]

In [ ]:
gdfs = []
for sat in sats:
    poly_df = forecast_fovs(sat, times, inst)
    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

## Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
poly_df['color'] = poly_df['id'].apply(cmap)

plot_df = poly_df[poly_df["lonspan"] < 20].copy()

plot_df.to_file("./tmp/fovs.geojson")
plot_df

In [ ]:
## Coverage data analysis for single satellite/ batch of satellites

aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
xcell_size = ycell_size = 0.1

# 1) Create a grid of equally spaced points
grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)

# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.to_file('./tmp/n_visits.geojson')
grid.n_visits.fillna(0).describe()



In [ ]:
## Plotting Revisit Map

## Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

## Create colormap and apply to img
## TODO: Make this our geoTIFF item for STAC catalog
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m

In [ ]:
# f = open("./historical_tles/sat39084.txt", "r")

# tles = []
# for x in f:
#   if x[0] == "1":
#         # print(x) 
#         tle1 = x
#   if x[0] == "2":
#         # print(x) 
#         tle2 = x
#         tle = tle1 + tle2
#         tles.append(tle)